In [1]:
from jax_circuits import *
from scipy.stats import unitary_group
import matplotlib.pyplot as plt

import time
import pickle

from qiskit import transpile
from qiskit.quantum_info import Operator

from topology import *

from jax import custom_jvp, jvp, grad
from jax.scipy.optimize import minimize

key = random.PRNGKey(42)

Number of ideas to try in optimization of Toffoli gates
- Push on the CP learning
- Try to use ancillas
- Try to find decompositions up to swaps

# Toffoli 3

In [2]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

## Randomized learning

In [3]:
%%time

num_attempts = 10
key = random.PRNGKey(13)
for _ in range(num_attempts):
    key, subkey = random.split(key)
    current_placements = random_placements(3, 6, key=subkey)
    anz = Ansatz(3, 'cz', placements={'free':current_placements})
    break
    ah, lh, sh = unitary_learn_repeat(anz.unitary, 
                                      u_toff3, 
                                      anz.num_angles, 
                                      'adam', 
                                      0.1, 
                                      num_repeats=5, 
                                      target_loss=1e-4)
    if sum(sh)>0:
        print('decomposition found')

CPU times: user 380 ms, sys: 7.9 ms, total: 388 ms
Wall time: 390 ms


In [25]:
print(type(current_placements[0][0]))
print(type(placements[1][1]))
print(type(int(current_placements[0][0])))

<class 'numpy.int32'>
<class 'int'>
<class 'int'>


In [22]:
int(current_placements)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [26]:
# placements = [[1, 2], [0, 1], [1, 0], [2, 0], [0, 2], [1, 2]]
# anz= Ansatz(3, 'cz', placements={'free': current_placements})
placements = [[int(i), int(j)] for i,j in list(current_placements)]
anz= Ansatz(3, 'cz', placements={'free': placements})
anz.circuit().draw()

┌────────┐┌────────┐┌────────┐                            ┌─────────┐»
q_0: ┤ RZ(a0) ├┤ RX(a1) ├┤ RZ(a2) ├──────────────────────────■─┤ RY(a13) ├»
     ├────────┤├────────┤├────────┤    ┌────────┐┌─────────┐ │ ├─────────┤»
q_1: ┤ RZ(a3) ├┤ RX(a4) ├┤ RZ(a5) ├─■──┤ RY(a9) ├┤ RX(a10) ├─■─┤ RY(a15) ├»
     ├────────┤├────────┤├────────┤ │ ┌┴────────┤├─────────┤   └─────────┘»
q_2: ┤ RZ(a6) ├┤ RX(a7) ├┤ RZ(a8) ├─■─┤ RY(a11) ├┤ RX(a12) ├──────────────»
     └────────┘└────────┘└────────┘   └─────────┘└─────────┘              »
«     ┌─────────┐   ┌─────────┐┌─────────┐   ┌─────────┐┌─────────┐   »
«q_0: ┤ RX(a14) ├─■─┤ RY(a19) ├┤ RX(a20) ├─■─┤ RY(a23) ├┤ RX(a24) ├─■─»
«     ├─────────┤ │ ├─────────┤├─────────┤ │ └─────────┘└─────────┘ │ »
«q_1: ┤ RX(a16) ├─■─┤ RY(a17) ├┤ RX(a18) ├─┼────────────────────────┼─»
«     └─────────┘   └─────────┘└─────────┘ │ ┌─────────┐┌─────────┐ │ »
«q_2: ─────────────────────────────────────■─┤ RY(a21) ├┤ RX(a22) ├─■─»
«                                            └─────────┘└─────────┘   »
«     ┌─────────┐┌─────────┐                         
«q_0: ┤ RY(a25) ├┤ RX(a26) ├─────────────────────────
«     └─────────┘└─────────┘   ┌─────────┐┌─────────┐
«q_1: ───────────────────────■─┤ RY(a29) ├┤ RX(a30) ├
«     ┌─────────┐┌─────────┐ │ ├─────────┤├─────────┤
«q_2: ┤ RY(a27) ├┤ RX(a28) ├─■─┤ RY(a31) ├┤ RX(a32) ├
«     └─────────┘└─────────┘   └─────────┘└─────────┘

## CP learning

With suitable hyperparameters finding optimal 6-gate decomposition is in fact rather easy.